In [3]:
import pandas as pd
import xlsxwriter

In [4]:
# read metrics and concat
df_dte = pd.read_excel("metrics/dte_metrics.xlsx")
df_nn = pd.read_excel("metrics/nn_metrics.xlsx")
df_log = pd.read_excel("metrics/log_metrics.xlsx")
df_svm = pd.read_excel("metrics/svm_metrics.xlsx")

df = pd.concat([df_dte, df_nn])
df = pd.concat([df, df_log])
df = pd.concat([df, df_svm])

df.to_excel("metrics/all_metrics.xlsx")
df.to_pickle("metrics/all_metrics.pkl")

In [5]:
# grouping
grouped = df.groupby(df.group)
df_abiEltern = grouped.get_group("abiEltern")
df_gender = grouped.get_group("gender")
df_erstsprache = grouped.get_group("erstsprache")
df_buecher = grouped.get_group("buecher")

In [6]:
def calculate_metrics(
    df, group_col, pp_col, eo_col, sa_col, pe_col, advantaged_group, disadvantaged_group
):
    """
    Function calculates fairness metrics pp, eo, sa and pe and return pivoted table
    """
    df = df.drop(columns=["group", "Unnamed: 0", "Accuracy"])
    df = pd.pivot_table(
        df,
        values=["Precision", "Recall", "AUC", "FPR"],
        index=["model"],
        columns=[group_col],
    )
    df[pp_col] = df.Precision[advantaged_group] - df.Precision[disadvantaged_group]
    df[eo_col] = df.Recall[disadvantaged_group] - df.Recall[advantaged_group]
    df[sa_col] = df.AUC[advantaged_group] - df.AUC[disadvantaged_group]
    df[pe_col] = df.FPR[disadvantaged_group] - df.FPR[advantaged_group]
    df = df.drop(columns=["AUC", "Precision", "Recall", "FPR"])
    df.columns = df.columns.droplevel(1)
    df = pd.pivot_table(df, values=[pp_col, eo_col, sa_col, pe_col], columns=["model"])

    return df


# gender
df_gender = calculate_metrics(
    df_gender, "subgroup", "PP", "EO", "SA", "PE", "girls", "boys"
)

# first language
df_erstsprache = calculate_metrics(
    df_erstsprache, "subgroup", "PP", "EO", "SA", "PE", "deutsch", "migration"
)

# HLE
df_buecher = calculate_metrics(
    df_buecher, "subgroup", "PP", "EO", "SA", "PE", "buch1", "buch0"
)

# parental education
df_abiEltern = calculate_metrics(
    df_abiEltern, "subgroup", "PP", "EO", "SA", "PE", "abi", "keinAbi"
)

In [7]:
"""
functions to format results
set two threshols: one at |0.02| in orange and one at |0.05| in red
format all negative values in bold
"""


def threshold001(v, props=""):
    return props if (v > 0.02) or (v < -0.02) else None


def threshold005(v, props=""):
    return props if (v > 0.05) or (v < -0.05) else None


def negativeValue(v, props=""):
    return props if (v < 0) else None


def showTable(df):
    styled = (
        df.style.set_properties(color="black", align="right")
        .set_properties(**{"background-color": "white"})
        .applymap(threshold001, props="color:orange;")
        .applymap(threshold005, props="color:red;")
        .applymap(negativeValue, props="font-weight:bold;")
    )
    return styled

In [8]:
s = showTable(df_gender)
s

model,DTE,LogReg,NN,SVM
EO,0.001718,-0.001130,0.000806,0.000096
PE,-0.001040,-0.004562,-0.003071,-0.003988
PP,0.000097,-0.000999,-0.000542,-0.000912
SA,-0.001379,-0.001716,-0.000180,-0.002042


In [9]:
s = showTable(df_erstsprache)
s

model,DTE,LogReg,NN,SVM
EO,0.005696,0.006497,0.005271,0.006917
PE,-0.002558,-0.003803,-0.002493,-0.002501
PP,-0.000427,-0.000812,-0.000388,-0.000470
SA,-0.004127,-0.005150,-0.000505,-0.004709


In [10]:
s = showTable(df_abiEltern)
s

model,DTE,LogReg,NN,SVM
EO,0.002680,0.004920,0.006950,0.008079
PE,-0.007158,-0.004749,-0.004681,-0.003271
PP,-0.001893,-0.001075,-0.001105,-0.000696
SA,-0.004919,-0.004835,-0.000863,-0.005675


In [11]:
s = showTable(df_buecher)
s

model,DTE,LogReg,NN,SVM
EO,-0.006524,-0.006835,-0.000830,-0.009373
PE,0.000948,0.000587,0.002306,-0.000007
PP,0.000816,0.000757,0.001309,0.000454
SA,0.003736,0.003711,0.000313,0.004683


In [12]:
# save as excel
writer = pd.ExcelWriter("fairness_evaluation.xlsx", engine="xlsxwriter")

df_gender.to_excel(writer, sheet_name="Gender")
df_abiEltern.to_excel(writer, sheet_name="AbiEltern")
df_erstsprache.to_excel(writer, sheet_name="ErstSprache")
df_buecher.to_excel(writer, sheet_name="Buecher")
writer.save()